In [2]:
!conda env list

# conda environments:
#
base                  *  C:\Users\daydr\Anaconda3
flask-aws                C:\Users\daydr\Anaconda3\envs\flask-aws
segmri                   C:\Users\daydr\Anaconda3\envs\segmri



In [50]:
import pyodbc
import pandas as pd
from pandas import merge

In [10]:
pyodbc.drivers()

['SQL Server',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'ODBC Driver 13 for SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

In [30]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-A3NFG0UM;DATABASE=S19SQLPlayground_Seb;Trusted_Connection=yes;')
cursor = conn.cursor()

In [129]:
sqlGetCustomers = "SELECT * FROM dbo.Customer;";
cursor.execute(sqlGetCustomers)
customersData = cursor.fetchall()
customersDf = pd.read_sql(sqlGetCustomers,conn)

sqlGetProducts = "SELECT * FROM dbo.Product;";
cursor.execute(sqlGetProducts)
productsData = cursor.fetchall()
productsDf = pd.read_sql(sqlGetProducts,conn)

sqlGetPurchases = "SELECT * FROM dbo.Purchase;";
cursor.execute(sqlGetPurchases)
purchasesData = cursor.fetchall()
purchasesDf = pd.read_sql(sqlGetPurchases,conn)

In [ ]:
conn.close()

In [130]:
# print(customersDf);print()
# print(productsDf);print()
# print(purchasesDf)

   CustomerId CustomerFirstName CustomerLastName
0           1         Sébastien       Corniglion
1           2               Léo          Souquet

   ProductId     ProductDescription  ProductUnitPrice
0          1         USB Stick 64GB              60.0
1          2            MacBook Pro            2500.0
2          3                   iPad             900.0
3          4  Laptop Windows 10 Pro            1500.0

   CustomerId  ProductId  Quantity
0           1          1         2
1           1          2         5
2           1          3         2
3           1          4         4
4           2          1         5


In [151]:
customersDf["key"] = 1; productsDf["key"]=1
custprodCartesianDf = merge(customersDf,productsDf,on='key')[['CustomerId', 'ProductId']]

In [152]:
custprodCartesianDf['Purchased'] = custprodCartesianDf.apply(tuple, 1).isin(purchasesDf[['CustomerId', 'ProductId']].apply(tuple, 1))

In [153]:
# custprodCartesianDf

,CustomerId,ProductId,Purchased
0,1,1,True
1,1,2,True
2,1,3,True
3,1,4,True
4,2,1,True
5,2,2,False
6,2,3,False
7,2,4,False


In [156]:
customersDf['NotPurchasedAll'] = customersDf[['CustomerId']].isin(custprodCartesianDf.query('Purchased==False')[['CustomerId']].values.ravel())

In [160]:
customersDf.query('NotPurchasedAll==False')[['CustomerId']].reindex()

,CustomerId
0,1
